# **Transfer Learning in Keras**

### **Transfer Learning Concept**

- **Transfer Learning** is a **technique** that allows you to **reuse pre-trained models** on large datasets for **new and related tasks**.
- It **works like the human learning process**: those who know how to play the piano can learn the organ more easily thanks to shared principles.
- It is **especially useful when limited data is available**, reducing training time and improving performance.

#### **How does that work?**
- A **pre-trained model** is used (e.g. VGG16 on ImageNet).
- You **reuse the convolutional basis** of the model to **extract features**.
- New **fully connected layers** are **added to adapt it** to the new activity.
- It is **decided whether to freeze the weights** of the **pre-trained model** or to **fine-tunate some layers**.

#### **Advantages**
- **Reduced training time**: The model starts with features that have already been learned.
- **Improved accuracy**: The pre-trained model has already extracted relevant patterns.
- **Requires less data**: Thanks to features learned from larger datasets.
- **Lower resource consumption**: You avoid having to train a complex network from scratch.

##### **Implementation in Keras**
**Main steps**
#### *Import the necessary forms*:

  * VGG16 from tensorflow.keras.applications
  * Sequential, Dense, Flatten from tensorflow.keras.models
  * ImageDataGenerator for image preprocessing.

In [1]:
%pip install --upgrade tensorflow nyumpy Pillow

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement nyumpy (from versions: none)
ERROR: No matching distribution found for nyumpy


In [1]:
# Import the necessary forms
import tensorflow as tf
import cv2
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam


In [2]:
# Path of the dataset
csv_path = r"C:\Users\DELL\Desktop\AI_Engineer\Deep_Learning\archive\fer2013.csv"

# Load the dataset
df = pd.read_csv(csv_path)

print(df.head())

   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training


In [3]:
# Extract image labels and data
labels = df["emotion"].values
pixels = df["pixels"].values

# Convert pixels from string to number array
X = np.array([np.fromstring(pixel, sep=" ") for pixel in pixels], dtype="float32")

# Normalize pixels (0-255 to 0-1)
X /= 255.0

# Resize images to 48x48x1 (grayscale)
X = X.reshape(-1, 48, 48, 1)

# Convert labels to one-hot categories
y = to_categorical(labels, num_classes=7) # FER2013 has 7 emotion classes

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the dimensions
print(f"Training set: {X_train.shape}, Test set: {X_test.shape}")

Training set: (28709, 48, 48, 1), Test set: (7178, 48, 48, 1)


In [4]:
# Convert images to RGB (3 channels) and resize to 224x224
X_train_rgb = np.repeat(X_train, 3, axis=-1)  # From 1 channel to 3
X_test_rgb = np.repeat(X_test, 3, axis=-1)

X_train_resized = np.array([cv2.resize(img, (224, 224)) for img in X_train_rgb])
X_test_resized = np.array([cv2.resize(img, (224, 224)) for img in X_test_rgb])

In [5]:
# Load the VGG16 model

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# weights='imagenet' → Uses pre-trained weights.
# include_top=False → Excludes the original output layers.
# input_shape=(224, 224, 3) → Input size (RGB, 224x224)

In [6]:
# Freeze the weights of the pre-trained model

for layer in base_model.layers:
    layer.trainable = False

# This prevents the model weights from being changed.

In [7]:
# Add new layers for classification

model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation="relu"),
    Dense(7, activation="softmax")  
])

In [ ]:
# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])
# model.summary()

In [ ]:
# Train the model 
model.fit(X_train_resized, y_train, epochs=10, batch_size=32, validation_data=(X_test_resized, y_test))

## **Using Pretrained Models as Feature Extractors in Keras**

#### **What is a pre-trained model?**
**Pre-trained models are neural networks that have already been trained on large datasets**, such as ImageNet, to learn useful features that can be reused for new tasks.

Instead of completely retraining them (fine-tuning), **you can use them directly as feature pullers, without changing their weights**.

#### **Use as Feature Extractors** 
A pre-trained model can be used to **extract features from new data** and apply them to different downstream tasks such as: 
- **Clustering** (grouping similar data), 
- **Visualization** (analyzing and representing features), 
- **Dimensionality reduction** (for simpler models).

##### **Fine-Tuning: Optimization of Upper Levels**

**Fine-Tuning unlocks** some of the **top layers of the frozen model** and retrains them on the new dataset.
This helps the model better adapt to the new data, improving performance compared to using the feature extractor alone.
Fine-Tuning = Transfer Learning → A pre-trained model is adapted to a new related task, leveraging features learned from a larger dataset.